In [47]:
from gemini import engine, helpers
import pandas as pd

df = pd.read_csv('BTCUSDT-4h-data.csv', parse_dates=['timestamp'])
df.columns = ['date', 'low', 'high', 'open', 'close', 'volume']
df

,date,low,high,open,close,volume
0,2017-08-17 04:00:00,4261.32000000,4349.99000000,4261.48000000,4349.99000000,82.08886500
1,2017-08-17 08:00:00,4333.32000000,4485.39000000,4333.32000000,4427.30000000,63.61988200
2,2017-08-17 12:00:00,4333.42000000,4485.39000000,4436.06000000,4352.34000000,174.56200100
3,2017-08-17 16:00:00,4200.74000000,4354.84000000,4352.33000000,4325.23000000,225.10971600
4,2017-08-17 20:00:00,4258.56000000,4369.69000000,4307.56000000,4285.08000000,249.76991300
...,...,...,...,...,...,...
8278,2021-05-30 16:00:00,35300.00000000,36284.44000000,35477.00000000,36077.59000000,9731.03431700
8279,2021-05-30 20:00:00,35500.00000000,36350.00000000,36077.59000000,35641.27000000,7743.92270400
8280,2021-05-31 00:00:00,34250.55000000,36000.00000000,35641.26000000,34489.57000000,14130.86534500
8281,2021-05-31 04:00:00,34153.84000000,35975.00000000,34489.58000000,35850.17000000,16153.46863900


In [48]:
df = df.loc[5000:8000]

In [49]:
backtest = engine.backtest(df)

In [50]:
import numpy as np
# Build mean reversion strategy
from talib.abstract import *

def bands(df, timeperiod=26, nbdevup=2.6, nbdevdn=2.6, matype=0):
    cols = ['high', 'low', 'open', 'close', 'volume']
    HLOCV = {key: df[key].values for key in df if key in cols}
    u, m, l = BBANDS(HLOCV, timeperiod=timeperiod, nbdevup=nbdevup, nbdevdn=nbdevdn, matype=matype)
    df['upper']  = u
    df['middle'] = m
    df['lower']  = l
    return df

def touches(df):
    df['touch_upper'] = df.high >= df.upper
    df['touch_lower'] = df.low  <= df.lower
    df['crossing_dn'] = (df.close < df.middle) & (df.open > df.middle)
    df['crossing_up'] = (df.close > df.middle) & (df.open < df.middle)
    return df

def logic(account, lookback):
    try:
        lookback = bands(lookback)
        lookback = touches(lookback)
        today = lookback.iloc[-1]
        
        # Selling
        if today.touch_upper:
            exit_price = today.upper
            for position in account.positions:  
                if position.type == 'long':
                    account.close_position(position=position, 
                                           percent=1.0, 
                                           current_price=exit_price)

        if today.crossing_up:
            exit_price = today.close
            for position in account.positions:  
                if position.type == 'long':
                    account.close_position(position=position, 
                                           percent=1.0, 
                                           current_price=exit_price)
                    
        # Buying
        if today.touch_lower | today.crossing_dn:
            risk          = 1
            entry_price   = today.lower
            entry_capital = account.buying_power*risk
            if entry_capital > 0:
                account.enter_position(type='long', 
                                       entry_capital=entry_capital, 
                                       entry_price=entry_price)
        if today.crossing_dn:
            risk          = 1
            entry_price   = today.close
            entry_capital = account.buying_power*risk
            if entry_capital > 0:
                account.enter_position(type='long', 
                                       entry_capital=entry_capital, 
                                       entry_price=entry_price)
    
    except Exception as e:
        print(e)
        pass # Handles lookback errors in beginning of dataset

In [51]:
import warnings
warnings.filterwarnings('ignore')

In [52]:
backtest.start(1000, logic)

,benchmark_equity,strategy_equity,benchmark_return,strategy_return
date,,,,
2019-12-01 04:00:00,7277.44000000,1000.00000000,NaN,NaN
2019-12-01 08:00:00,7313.80000000,1000.00000000,0.00499626,0.00000000
2019-12-01 12:00:00,7306.22000000,1000.00000000,-0.00103640,0.00000000
2019-12-01 16:00:00,7358.39000000,1000.00000000,0.00714049,0.00000000
2019-12-01 20:00:00,7390.89000000,1000.00000000,0.00441673,0.00000000
...,...,...,...,...
2021-04-13 16:00:00,62997.02000000,1000.00000000,-0.00528660,0.00000000
2021-04-13 20:00:00,63575.00000000,1000.00000000,0.00917472,0.00000000
2021-04-14 00:00:00,63749.19000000,1000.00000000,0.00273991,0.00000000


In [53]:
backtest.results()

-------------- Results ----------------

Buy and Hold : 785.29%
Net Profit   : 7852.92
Strategy     : 0.0%
Net Profit   : 0.0
Longs        : 0
Sells        : 0
Shorts       : 0
Covers       : 0
--------------------
Total Trades : 0

---------------------------------------


In [54]:
backtest.chart(True)